# Writing data to and reading data from a Database using Python

## Libraries and settings

In [2]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_02_KK


### 4f) Create a data base for the supermarket data

In [3]:
# Create data base
conn = sqlite3.connect('supermarkets_database.db')
cursor = conn.cursor()

# Load the JSON data from file
with open('supermarkets.json') as json_file:
    data = json.load(json_file)

# Use json_normalize to flatten the JSON and create a DataFrame
df = pd.json_normalize(data)

# Select columns
df = df[["id", "lat", "lon", "tags.brand", "tags.shop",
         "tags.addr:city", "tags.addr:street",
         "tags.addr:housenumber", "tags.addr:postcode"]]

# Rename columns
df = df.rename(columns={"tags.brand": "brand",
                        "tags.shop": "shop",
                        "tags.addr:city": "city",
                        "tags.addr:street": "street",
                        "tags.addr:housenumber": "housenumber",
                        "tags.addr:postcode": "postcode"})

# Create db table
cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (id INT(20),
                                                                 lat DECIMAL(8,2),
                                                                 lon DECIMAL(8,2),
                                                                 brand VARCHAR(50),
                                                                 shop VARCHAR(50),
                                                                 city VARCHAR(100),
                                                                 street VARCHAR(200),
                                                                 housenumber VARCHAR(50),
                                                                 postcode INT(4))''')

# Confirm changes to the table
conn.commit()

# Write data to table
df.to_sql(name='supermarkets_table',
          con=conn,
          index=False,
          if_exists='replace')


3392

### 4g) Write an SQL -query to filter all supermarkets in the city of Winterthur

In [4]:
# Open a connection to the data base 'apartment_database.db'
conn = sqlite3.connect('supermarkets_database.db') 

# List tables in supermarket_database.db
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print('Tables in supermarkets_database.db:\n', tables)

# List column names in supermarket_table
cursor = conn.execute('SELECT * FROM supermarkets_table')
colnames = list(map(lambda x: x[0], cursor.description))
print('\nColnames in supermarkets_table:\n', colnames)

# Query the SQL-table
print('\nSQL query')
cursor.execute('''SELECT * FROM supermarkets_table WHERE city = "Winterthur"''')
df = pd.DataFrame(cursor.fetchall(), 
                  columns=['id', 'lat', 'lon', 'brand', 'shop', 'city', 'street', 'housenumber', 'postcode'])

df.head()

Tables in supermarkets_database.db:
 [('supermarkets_table',)]

Colnames in supermarkets_table:
 ['id', 'lat', 'lon', 'brand', 'shop', 'city', 'street', 'housenumber', 'postcode']

SQL query


,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406
1,246533376,47.518953,8.720071,Migros,supermarket,Winterthur,Schaffhauserstrasse,152,8400
2,548919417,47.506863,8.712208,Migros,supermarket,Winterthur,Wülflingerstrasse,71,8400
3,677194765,47.503208,8.752131,None,supermarket,Winterthur,Frauenfelderstrasse,69,8404
4,706203439,47.500848,8.725938,None,supermarket,Winterthur,Bankstrasse,8/12,8400


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [5]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.2.0-1012-azure
Datetime: 2023-10-05 11:42:09
Python Version: 3.10.13
-----------------------------------
